Problem Approach: Resume REccomendation: 
we have 139 docx resume and we want to filter all to pick the best resume available for a job_description/opening.

1 We will use doc2txt to extract each resume

2 Perform feature engineering

3 use cosine similarity to compare with JD

4 loop through them each resume

5 lastly, tabulate each resume sort_by to extract the top resumes for JD and otherwise

In [0]:
#import packages
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob
import docx

In [9]:
!pip install python-docx

     |████████████████████████████████| 5.5MB 2.8MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=3212a4ca93d2d9964589fcec8449db3b713fd8076e93e284f4fd06f4a67c005c
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [12]:
#upload resume from drive
from google.colab import files
uploaded = files.upload()

Saving ABHINAV_RAHATE_CV.PDF to ABHINAV_RAHATE_CV.PDF
Saving aditya_Kashyn_ Resume.docx to aditya_Kashyn_ Resume.docx
Saving Akshay Mukundwar.pdf to Akshay Mukundwar.pdf
Saving Alok Roy_Resume_DS.doc to Alok Roy_Resume_DS.doc
Saving Anoosh_1year...docx to Anoosh_1year...docx
Saving Anup Vasanth - BA.docx to Anup Vasanth - BA.docx
Saving Balaji_Resume.docx to Balaji_Resume.docx
Saving CV-Ajitesh_Chauhan-converted.pdf to CV-Ajitesh_Chauhan-converted.pdf
Saving Divyansh_Kumar_Singh.pdf to Divyansh_Kumar_Singh.pdf
Saving Frenix_Rajiah_Resume-converted.pdf to Frenix_Rajiah_Resume-converted.pdf
Saving General Content.docx to General Content.docx
Saving Karan_bari.pdf to Karan_bari.pdf
Saving lda_model_sym_wiki.pkl to lda_model_sym_wiki.pkl
Saving Nikhil Resume.docx to Nikhil Resume.docx
Saving Parag.pdf to Parag.pdf
Saving Priya_DS_Resume.docx to Priya_DS_Resume.docx
Saving Priya_Resume(1).docx to Priya_Resume(1).docx
Saving Priya_Resume.docx to Priya_Resume.docx
Saving Pulkit_Srivastava.pdf

In [0]:
#define a function to extract text from any given .docx
def getTextdoc(filename):
  doc = docx.Document(filename)
  fullText = []
  for para in doc.paragraphs:
    fullText.append(para.text)
  return '\n'.join(fullText)


In [0]:
#get .docx files from a given path
worddocx = glob.glob(uploaded, './*.docx')

In [0]:
#extract text doc and compile them in a dataframe
li= []

for filenames in worddocx:
  content = getTextdoc(filenames)
  li.append(content)

df= pd.DataFrame(li, columns=['Contents'])

In [176]:
job_descr

'Data Analyst Job Description Template\nWe are looking to hire a Data Analyst to join our data team. You will take responsibility for managing our master data set, developing reports, and troubleshooting data issues. To do well in this role you need a very fine eye for detail, experience as a data analyst, and deep understanding of the popular data analysis tools and databases.\nData Analyst Responsibilities:\nManaging master data, including creation, updates, and deletion.\nManaging users and user roles.\nProvide quality assurance of imported data, working with quality assurance analyst if necessary.\nCommissioning and decommissioning of data sets.\nProcessing confidential data and information according to guidelines.\nHelping develop reports and analysis.\nManaging and designing the reporting environment, including data sources, security, and metadata.\nSupporting the data warehouse in identifying and revising reporting requirements.\nSupporting initiatives for data integrity and nor

In [0]:
#import text vectoriser
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [47]:
#sample to ensure it works
cv.fit_transform(df.iloc[2], job_descr)

<1x126 sparse matrix of type '<class 'numpy.int64'>'
	with 126 stored elements in Compressed Sparse Row format>

In [0]:
#import cosine_similarity test to check compare vectors
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
#check if it works----Fails
cosine_similarity(cv.fit_transform(df.iloc[2], job_descr))

cosine_similarity(cv.fit_transform(df.iloc[2], job_descr))[0][1] * 100

array([[1.]])

In [61]:
#uploaded job description
uploaded2= files.upload()

Saving Data Analyst Job Description.docx to Data Analyst Job Description.docx


In [0]:
#extract text from JD
job_descr = getTextdoc('Data Analyst Job Description.docx')

In [107]:
#try out--- success
cosine_similarity(cv.fit_transform([getTextdoc(worddocx[34]), job_descr]))

array([[1.        , 0.55142193],
       [0.55142193, 1.        ]])

In [0]:
#create a list of vectorised text extracted from .docx file. And loop through all  
cos_sim = []
for i in range(len(worddocx)):
  abc = cosine_similarity(cv.fit_transform([getTextdoc(worddocx[i]), job_descr]))
  cos_sim.append(abc)

In [0]:
#find similarity text between resume vector-form and job description vector-form
Similarity_Score = []
for i in range(len(cos_sim)):
  x = cos_sim[i][1][0] * 100
  Similarity_Score.append(x)

#df.append()
#for i in range(len(cos_sim)):
 # df.assign(Similarity_Score = round(cos_sim[i][0][1] * 100))

In [0]:
#affix Similarity_Score in dataframe
df = df.assign(Sim_Score= Similarity_Score )

In [175]:
#sort table by Sim_Score
df_final = df.sort_values('Sim_Score', ascending= False)
df_final.head()


,Contents,Sim_Score
38,A young Data Scientist with an analytical bent...,72.182575
121,Madhurima Sengupta\nE-Mail: sgmadhurima@gmail....,71.857194
119,TS VENKATA KRISHNAN\nE-Mail: venkey619@icloud....,71.667460
61,resume_76@gmail.com\n(948)-261-2460\nMelissa B...,71.150767
133,resume_92@gmail.com\n977 221 6451\nRuth Willis...,69.677285


In [0]:
df_final=df_final.reset_index(drop = True)

In [189]:
df_final

,index,Contents,Sim_Score
0,38,A young Data Scientist with an analytical bent...,72.182575
1,121,Madhurima Sengupta\nE-Mail: sgmadhurima@gmail....,71.857194
2,119,TS VENKATA KRISHNAN\nE-Mail: venkey619@icloud....,71.667460
3,61,resume_76@gmail.com\n(948)-261-2460\nMelissa B...,71.150767
4,133,resume_92@gmail.com\n977 221 6451\nRuth Willis...,69.677285
...,...,...,...
134,40,resume_3@gmail.com\n(438).752.1860\nLTS Softwa...,36.992268
135,75,resume_112@gmail.com\n(322).641.0480\nWilliam ...,34.624703
136,47,resume_58@gmail.com\n625.388.5724\nJoseph Youn...,29.343838
137,8,resume_34@gmail.com\n(521) 753 1354\nErik McCu...,19.734507


In [192]:
#get any resume in df based on index 
print(getTextdoc(worddocx[38]))

A young Data Scientist with an analytical bent of mind with an unending thirst to learn Analytics, machine learning and leverage the same to solve business problems.
SUMMARY
I am currently working as Engineer at Tata Consultancy Services. Having close to a year of experience in Data Science and specializes in analytical problem solving and machine learning. expertise in Python and R, has some exposure to big data tools such as Hive and Spark, visualizations tools such as Tableau.
I am result oriented approach with focus on efficient implementation and customer centricity. Im having capabilities of understanding the needs of the business, its strategic direction within no time and identifying initiatives that will allow a business to meet those strategic goals. proficient in formulating statistical and analytical methodology, building models, decision making, identifying trends and patterns with data. Passionate to utilize open source analytical tools such as R to build and deploy predi